In [1]:
# Importing necessary libraries
import pandas as pd
import torch
import re
from nltk.corpus import stopwords
from transformers import AutoModelForCausalLM, AutoTokenizer

# Importing configuration
from config import CONFIG

# Ensuring the necessary NLTK data is available
import nltk
#nltk.download('stopwords') #uncomment this line when not already installed

In [2]:
# Defining functions to remove stopwords and preprocess text
def remove_stopwords(text):
    """Remove stop words from the input text."""
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])

def preprocess_text(text):
    """Lowercase, remove punctuation, and remove stop words."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return remove_stopwords(text)

# Loading the tokenizer and model with error handling
try:
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["model_name"])
    model = AutoModelForCausalLM.from_pretrained(CONFIG["model_name"])
    CONFIG['eos_token_id'] = tokenizer.eos_token_id  # Set the EOS token ID
except OSError as e:
    print(f"Error: Failed to load the model or tokenizer. Check your model name or network connection: {e}")
    exit(1)
except Exception as e:
    print(f"An unexpected error occurred while loading the model: {e}")
    exit(1)

# Loading the dataset with error handling
try:
    data = pd.read_csv(CONFIG["dataset_path"], usecols=CONFIG["use_cols"])
    data.columns = ['Questions', 'Answers']
except FileNotFoundError:
    print("Error: The dataset file was not found. Please make sure the file path is correct.")
    exit(1)
except ValueError as ve:
    print(f"Error: Incorrect columns specified in usecols: {ve}")
    exit(1)
except Exception as e:
    print(f"An unexpected error occurred while loading the dataset: {e}")
    exit(1)

data['Questions'] = data['Questions'].fillna('').astype(str).apply(preprocess_text)
data['Answers'] = data['Answers'].fillna('').astype(str).apply(lambda x: x.strip())
qa_pairs = dict(zip(data['Questions'], data['Answers']))

'# Load the dataset and preprocess text\ndata = pd.read_csv(CONFIG["dataset_path"], usecols=CONFIG["use_cols"])\ndata.columns = [\'Questions\', \'Answers\']\ndata[\'Questions\'] = data[\'Questions\'].fillna(\'\').astype(str).apply(preprocess_text)\ndata[\'Answers\'] = data[\'Answers\'].fillna(\'\').astype(str).apply(lambda x: x.strip())\nqa_pairs = dict(zip(data[\'Questions\'], data[\'Answers\']))'

In [6]:
# Defining functions for rule-based responses and dataset responses
def rule_based_response(input_text):
    input_text = input_text.lower()
    if "heart attack" in input_text or "allergic reaction" in input_text:
        return "If you or anyone else around are in a medical emergency, Please call ambulance services immediately."
    return None

def get_dataset_response(input_text):
    input_text = input_text.lower().strip()
    return qa_pairs.get(input_text, None)

# Interactive chat function
def chat_interactive():
    global chat_history_ids
    chat_history_ids = None  # Initializing the chat history
    print("Chatbot is online. Type 'quit' to exit.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Chatbot is now offline.")
            break

        response = rule_based_response(user_input) or get_dataset_response(user_input)
        if not response:
            new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
            bot_input_ids = new_user_input_ids if chat_history_ids is None else torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
            chat_history_ids = model.generate(
                bot_input_ids,
                attention_mask=torch.ones_like(bot_input_ids),
                max_length=CONFIG["max_length"],
                pad_token_id=CONFIG['eos_token_id'],
                temperature=CONFIG["temperature"],
                top_k=CONFIG["top_k"],
                top_p=CONFIG["top_p"],
                no_repeat_ngram_size=2,
                do_sample=True,
                num_return_sequences=1
            )
            response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

        print("Chatbot: " + response)

# Starting the chat
chat_interactive()

Chatbot is online. Type 'quit' to exit.


You:  I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?


Chatbot: Suicide is a horrible thing to do.


You:  i think i have a heart attack


Chatbot: If you or anyone else around are in a medical emergency, Please call ambulance services immediately.


...

Chatbot: There's a lot of people who have suicidal thoughts. If you need someone to talk to, I would be glad to chat. Just PM me. lt 3


You:  thank you for that, now i feel a little more secure


Chatbot: that you're here


You:  quit


Chatbot is now offline.


1 example of the interaction:

Chatbot is online. Type 'quit' to exit.  
You: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?  
Chatbot: You can't, but you can try, if you believe in yourself.  
You: i think i have a heart attack  
Chatbot: If you or anyone else around are in a medical emergency, Please call ambulance services immediately.  
You: what should i do as first aid?  
Chatbot: Don't try to change your thoughts. Try to be a positive person. Make friends with everyone. Be yourself and your feelings  
You: ok,, what is the weather today?  
Chatbot: It's raining. I don't know it too. :  
You: quit  
Chatbot is now offline.